
![](https://binderhub.readthedocs.io/en/latest/_images/architecture.png)


## BinderHub

- Deploys the Binder service in the cloud.
- A thin layer that sits on top of **repo2docker** and **JupyterHub**, orchestrating their interactions and resolving URLs



### repo2docker

- Automatically builds a Docker image from a code repo given a configuration file
- The built docker image contains all of the code + data + resources listed in the repo

### What happens when a Binder link is clicked?

1. The link to the repository is resolved by BinderHub.

2. BinderHub searches for a Docker image relating to the provided reference (for example, git commit hash, branch or tag).

3. If a Docker image is not found, BinderHub requests resources from the Kubernetes cluster to run repo2docker to do the following:

 - Fetch the repository,

 - Build a Docker image containing the software requested in the configuration file,

 - Push that image to the Docker registry.

 - BinderHub sends the Docker image to JupyterHub.

4. JupyterHub requests resources from the Kubernetes cluster to serve the Docker image.

5. JupyterHub connects the user’s browser to the running Docker environment.

6. JupyterHub monitors the container for activity and destroys it after a period of inactivity.



## repo2singularity 

- A wrapper around repo2docker producing Jupyter enabled **Singularity** images.
- Relies on `repo2docker`, `docker`, and `singularity`


### There are 2 ways to build a singularity container via `repo2singularity`

1. Build a singularity container on a **Linux** system on which you have admin privilege
 - Requires having admin privilege
 - Requires having singularity, and docker installed


2. Build a singularity container on a remote builder
 - Doesn't require admin privilege
 - The remote builder must have singularity and docker installed

### Container Formats used by `repo2singularity`

- **squashfs**: 
    - The default container format
    - A compressed read-only file system
    
- **directory**:
  - Also known as `sandbox`
  - Standard Unix directory containing a root container image
  - Writable sandbox 

## How does repo2singularity work


```bash
# Build singularity image using the remote builder, retrieve it and cache it locally
repo2singularity --remote --endpoint-url http://ec2-18-236-169-165.us-west-2.compute.amazonaws.com:8000/api https://github.com/andersy005/singularity-binder
```

```bash
# Run the built image on our HPC system
repo2singularity --run https://github.com/andersy005/singularity-binder
```


[*Client side*] 

1. The link  to the repository is resolved by repo2singularity
  - If `--ref` (git commit hash, branch or tag) is specified, used it in conjuction with the repo url to construct an image name
  

2. Repo2singularity posts the following to the remote builder's HTTP endpoint

```json
{
  "url": "string",
  "ref": "string",
  "image_name": "string"
}
```

[*Server side: remote builder*]

3. Repo2singularity 

  - Fetchs the repository
  - Builds a Docker image (via `repo2docker`) containing the software requested in the configuration file
  - Builds a Singularity image (in **squashfs** format) using the built Docker image
  - Sends the built singularity image back to the client side
  
 
[*Client side*]

4. Repo2singularity takes the returned singularity image file and converts it into a writable directory (**sandbox** format)

5. Lauches the singularity image



In [1]:
which python

/glade/work/abanihi/softwares/miniconda3/bin/python


In [2]:
lsb_release --all

LSB Version:	n/a
Distributor ID:	SUSE
Description:	SUSE Linux Enterprise Server 12 SP4
Release:	12.4
Codename:	n/a


In [3]:
whoami

abanihi


In [4]:
cat /proc/1/cgroup

12:net_cls,net_prio:/init.scope
11:devices:/init.scope
10:perf_event:/
9:cpu,cpuacct:/init.scope
8:cpuset:/
7:blkio:/init.scope
6:memory:/init.scope
5:hugetlb:/
4:rdma:/
3:pids:/init.scope
2:freezer:/
1:name=systemd:/init.scope
